### Build the dataframe from the raw files

In [ ]:
# Import required packages
import pandas as pd
import os
import xml.etree.ElementTree as ET
import spacy
import re

In [ ]:
def parse_xml(xml_file):
    global empty_counter

    # Load the XML file
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # Define the namespace
    namespace = {
        'atom': 'http://www.w3.org/2005/Atom',
        'nitf': 'http://iptc.org/std/NITF/2006-10-18/',
        'dc': 'http://purl.org/dc/elements/1.1/'
    }

    # Extract the ID
    nexis_id = root.find('.//atom:id', namespace).text
    nexis_id = nexis_id.split('Item:')[1]
    nexis_id = nexis_id.replace("-", "")

    # Extract the title
    title = root.find('.//atom:title', namespace).text

    # Extract the published date
    published_date = root.find('.//atom:published', namespace).text
    published_date = published_date.split('T')[0]

    # Extract the publisher
    publisher = root.find('.//metadata/publicationInfo/publicationName').text


    # Extract the body text
    body_text = root.find('.//nitf:body.content//bodyText//p', namespace)

    # Extract all body paragraphs and join them into a single line
    body_paragraphs = []
    body_text_element = root.find('.//nitf:body.content//bodyText', namespace)
    if body_text_element is not None:
        for p_element in body_text_element.findall('.//p', namespace):
            if p_element.text:
                body_paragraphs.append(p_element.text.strip())
    text = ' '.join(body_paragraphs)

    # Check for empty text bodies and return
    if text == '':
        #print('No text body in file ' + file_path)
        empty_counter += 1
        return

    # Create a dictionary with the extracted values
    data_dict = {
        'nexis_id': [nexis_id],
        'title': [title],
        'publication_date': [published_date],
        'publisher': [publisher],
        'text': [text]
    }

    df = pd.DataFrame(data_dict)
    return df

In [ ]:
# Root Directory of the data
folder_path = 'c:\\Friendcloud\\_University\\_Masterarbeit\\Data\\Nexis\\'

# Initialize empty counter
empty_counter = 0

# Go through all subdirectories and fill raw_data with the data
raw_data = []
for root, _, files in os.walk(folder_path):
    for file in files:
        if file.endswith('.xml'):
            try:
                file_path = os.path.join(root, file)
                data = parse_xml(file_path)
                raw_data.append(data)
            except:
                print('Error in file ' + file_path)

# Build pandas DataFrame
df_raw = pd.concat(raw_data, ignore_index=True)
print('Number of articles removed: ' + str(empty_counter))
print('Number of articles accepted: ' + str(len(df_raw.index)))

In [ ]:
df_raw

In [ ]:
df_raw.to_pickle("../Data/df_raw.pkl")

### Basic Data preprocessing

In [ ]:
df_processed = pd.read_pickle("../Data/df_raw.pkl")

In [ ]:
# Add year of publication to dataset
df_processed['publication_date'] = pd.to_datetime(df_processed['publication_date'], errors='coerce')
df_processed['publication_year'] = df_processed['publication_date'].dt.year
df_processed = df_processed[['nexis_id', 'title', 'publication_date', 'publication_year', 'publisher', 'text']]

In [ ]:
# Remove articles with duplicate id
count_before = len(df_processed.index)
df_processed = df_processed.drop_duplicates(subset='nexis_id', keep='first')
count_after = len(df_processed.index)
print('Number of articles removed: ' + str(count_before - count_after))

In [ ]:
# Find examples of articles with duplicate text
df_processed[df_processed.duplicated(subset='text', keep=False)].sample(10)

In [ ]:
# Remove articles with duplicate text
count_before = len(df_processed.index)
df_processed = df_processed.drop_duplicates(subset='text', keep='first')
count_after = len(df_processed.index)
print('Number of articles removed: ' + str(count_before - count_after))

In [ ]:
# Remove articles with length less than 100 characters
count_before = len(df_processed.index)
df_processed = df_processed[df_processed['text'].str.len() > 100]
count_after = len(df_processed.index)
print('Number of articles removed: ' + str(count_before - count_after))

In [ ]:
# Reindex the dataframe
df_processed = df_processed.reset_index(drop=True)

In [ ]:
df_processed

In [ ]:
df_processed.to_pickle("../Data/df.pkl")

### Text Clean up

In [84]:
VARIANT = "SMALL" # "SMALL", "MEDIUM" , "FULL"

if VARIANT == "SMALL":
    df_processed = pd.read_pickle("../Data/df.pkl")
    df_processed = df_processed.head(20)
elif VARIANT == "MEDIUM":
    df_processed = pd.read_pickle("../Data/df.pkl")
    df_processed = df_processed.head(500)
else:
    df_processed = pd.read_pickle("../Data/df.pkl")

In [85]:
df_processed

,nexis_id,title,publication_date,publication_year,publisher,text
0,5MNMH621JB0GF09H0000000,Angst vor dem «harten Brexit» auf der Insel - ...,2017-01-15,2017,AP Deutsch,Wenn Großbritannien Ende März den Ausstieg aus...
1,5SM3THB1DXFJ50MP0000000,Fleischindustrie wehrt sich gegen Marketing fü...,2018-06-21,2018,AP Deutsch,Fleisch aus dem Labor ist noch eine Zukunftsvi...
2,5SM3THB1DXFJ50MY0000000,Der AP-Überblick am Nachmittag,2018-06-21,2018,AP Deutsch,Die AP Weltnachrichten haben heute unter ander...
3,5STNVWH1DXFJ53VM0000000,Laborfleisch soll in drei Jahren auf die Telle...,2018-07-17,2018,AP Deutsch,Maastricht (AP) - Das niederländische Unterneh...
4,5VHK2XG1JB0GF4Y50000000,Israelische Forscher wollen künstliche Steaks ...,2019-02-26,2019,AP Deutsch,"Die Weltbevölkerung wächst, die Nachfrage nach..."
5,5WJBFTV1JB0GF2P80000000,Fleisch ohne Schlachten,2019-07-12,2019,AP Deutsch,Frisch aus dem Labor: In den USA stellen Start...
6,5WJD56M1DXFJ54SK0000000,Der AP-Überblick am Nachmittag,2019-07-12,2019,AP Deutsch,Die AP Weltnachrichten haben heute unter ander...
7,6353TNT1DXFJ54180000000,Immer mehr Start-ups wollen Insekten auf unser...,2021-07-16,2021,AP Deutsch,Mehrere Dutzend europäische Unternehmen sehen ...
8,63MMKPN1JB0GF3X10000000,Prinz William gibt Finalisten für Umweltpreis ...,2021-09-17,2021,AP Deutsch,Der britische Prinz William hat die 15 Finalis...
9,5G96X4G1JB16301D0000000,Bundesrat fordert mehr Diktatur,2015-06-25,2015,Appenzeller Zeitung,Bundesrat Johann Schneider-Ammann fordert eine...


In [86]:
# Load the German language model in Spacy
nlp = spacy.load('de_core_news_lg')

# Define a function to clean the text
def clean_text(text):
    # Remove special characters
    text = re.sub(r"[^A-Za-z0-9äöüÄÖÜß ]+", '', text)
    
    # Lemmatize the text
    doc = nlp(text)
    lemmatized_text = ' '.join([token.lemma_ for token in doc])

    print(doc[0], ' ', doc[0].ent_type_)

    # If the first word has the label "GPE", remove it
    if doc[0].ent_type_ == 'GPE':
        print('GPE found')
        lemmatized_text = lemmatized_text.split(' ', 1)[1]
        
    return lemmatized_text

# Apply the clean_text function to the 'text' column in df_processed and save as a new dataframe df_clean
df_clean = df_processed.copy()
df_clean['text'] = df_clean['text'].apply(clean_text)

Wenn   
Fleisch   MISC
Die   
Maastricht   ORG
Die   
Frisch   PER
Die   
Mehrere   
Der   
Bundesrat   
Ryoji   
WASHINGTON   ORG
BRÜSSEL   LOC
Es   
Es   
Die   
Landwirtschaft   
Wolfhalden   LOC
Konsumverhalten   
Lebensmittel   


In [87]:
df_clean

,nexis_id,title,publication_date,publication_year,publisher,text
0,5MNMH621JB0GF09H0000000,Angst vor dem «harten Brexit» auf der Insel - ...,2017-01-15,2017,AP Deutsch,wenn Großbritannien Ende März der Ausstieg aus...
1,5SM3THB1DXFJ50MP0000000,Fleischindustrie wehrt sich gegen Marketing fü...,2018-06-21,2018,AP Deutsch,Fleisch aus der Labor sein noch ein Zukunftsvi...
2,5SM3THB1DXFJ50MY0000000,Der AP-Überblick am Nachmittag,2018-06-21,2018,AP Deutsch,der AP Weltnachrichten haben heute unter ander...
3,5STNVWH1DXFJ53VM0000000,Laborfleisch soll in drei Jahren auf die Telle...,2018-07-17,2018,AP Deutsch,Maastricht AP der niederländisch Unternehmen...
4,5VHK2XG1JB0GF4Y50000000,Israelische Forscher wollen künstliche Steaks ...,2019-02-26,2019,AP Deutsch,der Weltbevölkerung wachsen der Nachfrage nach...
5,5WJBFTV1JB0GF2P80000000,Fleisch ohne Schlachten,2019-07-12,2019,AP Deutsch,frisch aus der Labor in der USA stellen Startu...
6,5WJD56M1DXFJ54SK0000000,Der AP-Überblick am Nachmittag,2019-07-12,2019,AP Deutsch,der AP Weltnachrichten haben heute unter ander...
7,6353TNT1DXFJ54180000000,Immer mehr Start-ups wollen Insekten auf unser...,2021-07-16,2021,AP Deutsch,mehrere Dutzend europäisch Unternehmen sehen i...
8,63MMKPN1JB0GF3X10000000,Prinz William gibt Finalisten für Umweltpreis ...,2021-09-17,2021,AP Deutsch,der britisch Prinz William haben der 15 Finali...
9,5G96X4G1JB16301D0000000,Bundesrat fordert mehr Diktatur,2015-06-25,2015,Appenzeller Zeitung,Bundesrat Johann SchneiderAmmann fordern ein Z...
